# How do I _copy_ an app from the _public apps_?
### Overview
Apps can be copied from one of two sources:
1. other projects you are a member of
2. public reference. 

Here we focus on copying an app from _public reference_. 
 
### Prerequisites
 1. You need to be a member (or owner) of _at least one_ project.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly and pick one here).
 4. You understand how to <a href="apps_listAll.ipynb" target="_blank">list</a> apps within your project (we will just use that call directly and pick one here).
 
## Imports
We import the _Api_ class from the official sevenbridges-python bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The _Api_ object needs to know your **auth\_token** and the correct path. Here we assume you are using the .sbgrc file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify platform {cgc, sbg}
prof = 'cgc'

config_config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_config_file)

## Copy a Public App
We will first list all our projects, then list the apps within the Public Reference project<sup>1</sup>, and finally copy an app between the Public Reference to the my_project. Here we also explicitly set _'limit':100_ inside the _query_. This helps speed up the auto-pagination feature within the object constructor.

The critical information for this POST is the **app_id**. Note, you are **not** allowed to copy the same app **and** assign the same name more than once. If you change the name, it is ok.

To make these results very obvious, use an empty project as your MY\_PROJECT (e.g. your <a href=projects_makeNew.ipynb> cookbook example project</a>) or change the _name_ in the _data_ dictionary to something like 'Dept of Awesome'<sup>2</sup>

<sup>1</sup> Unlike files, apps are accesssible **both** *within* a project (here the Public Reference project) **and** by a *visibility* property (which may be set to 'public')

#### Why am I getting an Error Code 6000?
You found a known bug, we are _very sorry_ for this. It is already being worked on and should be ok soon. Try copying the app named **'Torrent Mapping Alignment Program Map4'**, this will work. Known affected workflows:
* Exome Coverage QC 1.0
* RNA-Seq De Novo Assembly and Analysis - Trinity 2.0.6
* RNA-Seq De Novo Assembly - Trinity 2.0.6
* Whole Exome Sequencing GATK 2.3.9.-lite

In [ ]:
# [USER INPUT] Set project name; project (p_) and file (f_) indices here:
my_project_name = 'Keep on Smiling'
a_name = 'Torrent Mapping Alignment Program Map4'


# pull out target project
my_project = [p for p in api.projects.query(limit=100).all() \
              if p.name == my_project_name]

# Double-check that target project exists
if not my_project:
    print('Target project (%s) not found, check spelling' % my_project_name)
    raise KeyboardInterrupt
else:
    my_project = my_project[0]
    
my_apps = api.apps.query(project = my_project.id, limit=100)
my_app_source = [a for a in api.apps.query(visibility='public', limit=100) \
                 if a.name == a_name][0]

duplicate_app = [a for a in my_apps.all() if a.name == my_app_source.name]

if duplicate_app:
    print('App already exists in second project, please try another app')
else:
    print('App (%s) does not exist in Project (%s); copying now' % \
          (a_name, my_project.name))
    
    my_new_app = my_app_source.copy(project = my_project.id, name = a_name)
        
    # re-list apps in target project to verify the copy worked
    my_apps = api.apps.query(project = my_project.id, limit=100)
    my_app_names = [a.name for a in my_apps.all()]
    
    if a_name in my_app_names:
        print('Sucessfully copied one app!')
    else:
        print('Something went wrong...')

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.cancergenomicscloud.org/docs/copy-an-app-secondary-method)